In [20]:
import requests
import time
import base64
import json

In [2]:
# Authentication
from settings import URI, AUTH

username, password = AUTH
credentials = f"{username}:{password}"

# Base64 encode the credentials
credentials_b64 = base64.b64encode(credentials.encode()).decode()

# Define the Authorization header
headers = {
  "Authorization": f"Basic {credentials_b64}"
}

In [268]:
# Define the Neo4j endpoint
endpoint = "http://localhost:7474/db/data/transaction/commit"

session = requests.Session()

def get_word_rows(json_response):
    return  [(data['row'][0][-3]['value'], data['row'][0][-1]['value']) for data in json_response["results"][0]['data']]

def get_word_paths(session, value, path_length):
    # Define the Neo4j query
    query = {
      "statements": [
        {
          "statement": f"MATCH (w:Word) USING TEXT INDEX w:Word(value) WHERE w.value = '{value}' MATCH p = (w)-[:HAS_WORD*1..{path_length}]->(:Word) WITH collect(distinct nodes(p)) AS uniqueNode UNWIND uniqueNode AS n RETURN n;",
          "resultDataContents": ["row"]
        }
      ]
    }

    # Send the request to the Neo4j endpoint
    request_time = time.time()
    response = session.post(endpoint, json=query, headers=headers)
    print(time.time() - request_time)
    # Print the response
    return response.json()

def get_word_data(session, value, path_length):
    json_response_time = time.time()
    word_path = get_word_paths(session, value, path_length)
    print(time.time() - json_response_time)
    return get_word_rows(word_path)

In [275]:
path = get_word_paths(session, "defenestration", 3)

0.4757564067840576


In [276]:
path["errors"]

[]

In [277]:
len(path["results"][0]["data"])

41586

In [224]:
# Call the function with the parameters
start = time.time()
word_path = get_word_data(session, "defenestration", 2)
print(time.time() - start)

2.104249954223633
2.104249954223633


IndexError: list index out of range

In [37]:
word_path

[('defenestration', 'thing'),
 ('thing', 'fashionable'),
 ('thing', 'mild'),
 ('thing', 'dressing'),
 ('thing', 'possession'),
 ('thing', 'act'),
 ('thing', 'something'),
 ('thing', 'product'),
 ('thing', 'usually'),
 ('thing', 'material'),
 ('thing', 'observation'),
 ('thing', 'imply'),
 ('thing', 'not'),
 ('thing', 'being'),
 ('thing', 'living'),
 ('thing', 'talk'),
 ('thing', 'appeal'),
 ('thing', 'precisely'),
 ('thing', 'situation'),
 ('thing', 'proper'),
 ('thing', 'separate'),
 ('thing', 'specialty'),
 ('thing', 'general'),
 ('thing', 'from'),
 ('thing', 'article'),
 ('thing', 'to'),
 ('thing', 'write'),
 ('thing', 'designate'),
 ('thing', 'accomplishment'),
 ('thing', 'activity'),
 ('thing', 'of'),
 ('thing', 'an'),
 ('thing', 'distinguished'),
 ('thing', 'concern'),
 ('thing', 'news'),
 ('thing', 'appearance'),
 ('thing', 'spatial'),
 ('thing', 'obsession'),
 ('thing', 'the'),
 ('thing', 'make'),
 ('thing', 'entity'),
 ('thing', 'own'),
 ('thing', 'concrete'),
 ('thing', 'circ

In [14]:
word_path.keys()

dict_keys(['results', 'errors'])

In [18]:
word_path['results']

[{'columns': ['p'],
  'data': [{'row': [[{'value': 'defenestration'}, {}, {'value': 'thing'}]],
    'meta': [[{'id': 265787, 'type': 'node', 'deleted': False},
      {'id': 781047, 'type': 'relationship', 'deleted': False},
      {'id': 187219, 'type': 'node', 'deleted': False}]],
    'graph': {'nodes': [{'id': '187219',
       'labels': ['Word'],
       'properties': {'value': 'thing'}},
      {'id': '265787',
       'labels': ['Word'],
       'properties': {'value': 'defenestration'}}],
     'relationships': [{'id': '781047',
       'type': 'HAS_WORD',
       'startNode': '265787',
       'endNode': '187219',
       'properties': {}}]}},
   {'row': [[{'value': 'defenestration'},
      {},
      {'value': 'thing'},
      {},
      {'value': 'fashionable'}]],
    'meta': [[{'id': 265787, 'type': 'node', 'deleted': False},
      {'id': 781047, 'type': 'relationship', 'deleted': False},
      {'id': 187219, 'type': 'node', 'deleted': False},
      {'id': 1756252, 'type': 'relationship', 

In [5]:
data_tuples = [(data['row'][0][-3]['value'], data['row'][0][-1]['value']) for data in word_path["results"][0]['data']]

TypeError: list indices must be integers or slices, not str

## MsgSpec Practice

In [5]:
from msgspec.json import decode
from msgspec import Struct

In [6]:
class Value(Struct):
    word: str
        
class RowItem(Struct):
    value: Value

class Row(Struct):
    row: list[list[dict]]

class Data(Struct):
    data: list[Row]

class Result(Struct):
    results: list[Data]

In [298]:
def get_word_paths_raw(session, value, path_length):
    # Define the Neo4j query
    query = {
      "statements": [
        {
          "statement": f"MATCH (w:Word) USING TEXT INDEX w:Word(value) WHERE w.value = '{value}' MATCH p = (w)-[:HAS_WORD*1..{path_length}]->(:Word) RETURN p;",
          "resultDataContents": ["row"]
        }
      ]
    }

    # Send the request to the Neo4j endpoint
    response_time = time.time()
    response = session.post(endpoint, json=query, headers=headers)
    print(time.time() - response_time)
    return response.content

def get_word_pairs_raw(session, value, path_length):
    # Define the Neo4j query
    query = {
      "statements": [
        {
          "statement": f"MATCH (w:Word) USING TEXT INDEX w:Word(value) WHERE w.value = '{value}' MATCH (w:Word)-[:HAS_WORD]->(w2:Word) MATCH (w2:Word)-[:HAS_WORD]->(w3:Word) RETURN w.value, w2.value, w3.value;",
          "resultDataContents": ["row"]
        }
      ]
    }

    # Send the request to the Neo4j endpoint
    response_time = time.time()
    response = session.post(endpoint, json=query, headers=headers)
    print(time.time() - response_time)
    return response.json()


def spec_word_rows(spec_response):
    return [(data.row[0][-3]['value'], data.row[0][-1]['value']) for data in spec_response.results[0].data]

In [300]:
raw_pairs = get_word_pairs_raw(session, "defenestration", 2)

0.03452944755554199


In [301]:
raw_pairs

{'results': [{'columns': ['w.value', 'w2.value', 'w3.value'],
   'data': [{'row': ['defenestration', 'throw', 'sense'],
     'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'switch'], 'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'in'], 'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'birth'], 'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'move'], 'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'any'], 'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'quickly'],
     'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'hastily'],
     'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'deposit'],
     'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'more'], 'meta': [None, None, None]},
    {'row': ['defenestration', 'throw', 'suddenly'],
     'meta': [None, None, None]},
    {'row': ['defenestration', 't

In [23]:
raw_path = get_word_paths_raw(session, "defenestration", 2)

In [110]:
type(raw_path)

bytes

In [289]:
spec_time = time.time()
raw_path = get_word_paths_raw(session, "run", 3)
raw_data = decode(raw_path, type=Result)
rows = spec_word_rows(raw_data)
print(time.time() - spec_time)

4.4449803829193115
7.293426513671875


In [179]:
# Depth: 1
len(rows)

17

In [181]:
# Depth: 2
len(rows)

890

In [291]:
# Depth: 3
len(set(rows))

99756

In [186]:
# Depth: 4
len(set(rows))

158100

In [169]:
json_time = time.time()
json_data = get_word_data(session, "defenestration", 4)
print(time.time() - json_time)

0.0120086669921875
0.016512393951416016
0.017012834548950195


In [105]:
json_data["results"][0]['data'][0]['row'][0][0]

{'value': 'defenestration'}

In [121]:
raw_data.results[0].data[0].row[0][0]['value']

'defenestration'

In [141]:
data_tuples = [(data.row[0][-3]['value'], data.row[0][-1]['value']) for data in raw_data.results[0].data]

In [172]:
len(data_tuples)

41586

In [194]:
raw_data.results[0].data

TypeError: unhashable type: 'Row'

TypeError: unhashable type: 'Row'

In [197]:
len(set(rows))

25182

## Putting it together to form Cytoscape Data

In [7]:
from neo4j_manager import Neo4jHTTPManager

In [8]:
database = Neo4jHTTPManager()

In [9]:
data = database.get_word_paths_raw('defenestration', 2)

In [21]:
json.loads(data)

{'results': [{'columns': ['p'],
   'data': [{'row': [[{'value': 'defenestration'}, {}, {'value': 'thing'}]],
     'meta': [[{'id': 265787, 'type': 'node', 'deleted': False},
       {'id': 781047, 'type': 'relationship', 'deleted': False},
       {'id': 187219, 'type': 'node', 'deleted': False}]]},
    {'row': [[{'value': 'defenestration'},
       {},
       {'value': 'thing'},
       {},
       {'value': 'fashionable'}]],
     'meta': [[{'id': 265787, 'type': 'node', 'deleted': False},
       {'id': 781047, 'type': 'relationship', 'deleted': False},
       {'id': 187219, 'type': 'node', 'deleted': False},
       {'id': 1756252, 'type': 'relationship', 'deleted': False},
       {'id': 269534, 'type': 'node', 'deleted': False}]]},
    {'row': [[{'value': 'defenestration'},
       {},
       {'value': 'thing'},
       {},
       {'value': 'mild'}]],
     'meta': [[{'id': 265787, 'type': 'node', 'deleted': False},
       {'id': 781047, 'type': 'relationship', 'deleted': False},
       {'id

In [10]:
decoded_data = decode(data, type=Result)
rows_with_length = database.spec_word_rows_with_length(decoded_data)

In [11]:
rows_with_length

[('defenestration', 'thing', 2.0),
 ('thing', 'fashionable', 3.0),
 ('thing', 'mild', 3.0),
 ('thing', 'dressing', 3.0),
 ('thing', 'possession', 3.0),
 ('thing', 'act', 3.0),
 ('thing', 'something', 3.0),
 ('thing', 'product', 3.0),
 ('thing', 'usually', 3.0),
 ('thing', 'material', 3.0),
 ('thing', 'observation', 3.0),
 ('thing', 'imply', 3.0),
 ('thing', 'not', 3.0),
 ('thing', 'being', 3.0),
 ('thing', 'living', 3.0),
 ('thing', 'talk', 3.0),
 ('thing', 'appeal', 3.0),
 ('thing', 'precisely', 3.0),
 ('thing', 'situation', 3.0),
 ('thing', 'proper', 3.0),
 ('thing', 'separate', 3.0),
 ('thing', 'specialty', 3.0),
 ('thing', 'general', 3.0),
 ('thing', 'from', 3.0),
 ('thing', 'article', 3.0),
 ('thing', 'to', 3.0),
 ('thing', 'write', 3.0),
 ('thing', 'designate', 3.0),
 ('thing', 'accomplishment', 3.0),
 ('thing', 'activity', 3.0),
 ('thing', 'of', 3.0),
 ('thing', 'an', 3.0),
 ('thing', 'distinguished', 3.0),
 ('thing', 'concern', 3.0),
 ('thing', 'news', 3.0),
 ('thing', 'appeara

In [34]:
dir(database)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'endpoint',
 'get_word_data',
 'get_word_paths_raw',
 'headers',
 'session',
 'spec_word_rows']

In [15]:
initial_value = "defenestration"
word_data = database.get_word_data(initial_value, 1)
# Nodes
nodes = [initial_value]
for word_tuple in word_data:
    nodes.append(word_tuple[1])

cyto_nodes = [
    {
        'data': {'id': word, 'label': word}
    }
    for word in nodes
]

# Edges
cyto_edges = [
    {'data': {'source': source, 'target': target}}
    for source, target in word_data
]

elements = cyto_nodes + cyto_edges


In [16]:
word_data

[('defenestration', 'thing'),
 ('defenestration', 'office'),
 ('defenestration', 'usually'),
 ('defenestration', 'political'),
 ('defenestration', 'from'),
 ('defenestration', 'out'),
 ('defenestration', 'party'),
 ('defenestration', 'window'),
 ('defenestration', 'expulsion'),
 ('defenestration', 'of'),
 ('defenestration', 'person'),
 ('defenestration', 'as'),
 ('defenestration', 'or'),
 ('defenestration', 'dismissal'),
 ('defenestration', 'a'),
 ('defenestration', 'swift'),
 ('defenestration', 'throw')]

In [5]:
print(type(elements))

<class 'list'>


In [9]:
nodes

['defenestration',
 'thing',
 'fashionable',
 'mild',
 'dressing',
 'possession',
 'act',
 'something',
 'product',
 'usually',
 'material',
 'observation',
 'imply',
 'not',
 'being',
 'living',
 'talk',
 'appeal',
 'precisely',
 'situation',
 'proper',
 'separate',
 'specialty',
 'general',
 'from',
 'article',
 'to',
 'write',
 'designate',
 'accomplishment',
 'activity',
 'of',
 'an',
 'distinguished',
 'concern',
 'news',
 'appearance',
 'spatial',
 'obsession',
 'the',
 'make',
 'entity',
 'own',
 'concrete',
 'circumstance',
 'utensil',
 'clothing',
 'spoken',
 'as',
 'detail',
 'effort',
 'a',
 'way',
 'equipment',
 'notion',
 'particular',
 'sphere',
 'inanimate',
 'state',
 'forte',
 'kind',
 'and',
 'purpose',
 'behave',
 'event',
 'object',
 'individual',
 'strong',
 'its',
 'matter',
 'within',
 'in',
 'point',
 'for',
 'or',
 'work',
 'substance',
 'aim',
 'distinct',
 'information',
 'phobia',
 'specify',
 'quality',
 'possessed',
 'piece',
 'especially',
 'capable',
 's